In [1]:
%load_ext autoreload
%autoreload 2

In [66]:
# Standard library imports
from typing import Union, Optional, Dict, Tuple
from pathlib import Path
import pickle
import pandas as pd
import xarray as xr
from helpers import cached_eval_run


def combine_datasets(results: Dict[int, Dict[str, Dict[str, Union[str, pd.DataFrame]]]]) -> pd.DataFrame:
    """
    Combine datasets from multiple folds.

    Parameters
    ----------
    results : Dict[int, Dict[str, Dict[str, Union[str, pd.DataFrame]]]]
        Dictionary containing results for different folds.

    Returns
    -------
    pd.DataFrame
        Combined DataFrame.
    """

    folds = results.keys()
    for fold in folds:
        for k,v in results[fold].items():
            basins.append(k)
            data_values.append(v['1D']['xr'])

    basins = [basin for fold in results.values() for basin in fold]
    data_values = [entry['1D']['xr'] for fold in results.values() for entry in fold.values()]
    combined_dataset = xr.concat(data_values, dim='basin').assign_coords(basin=basins)
    return combined_dataset.to_dataframe()

def create_pivot_tables(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Create pivot tables for simulated and observed streamflow.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the streamflow data.

    Returns
    -------
    Tuple[pd.DataFrame, pd.DataFrame]
        Tuple containing pivot tables for simulated and observed streamflow.
    """
    pivot_table_simulated = df.pivot_table(values='streamflow_mmd_sim', index='date', columns='basin')
    pivot_table_observed = df.pivot_table(values='streamflow_mmd_obs', index='date', columns='basin')
    return pivot_table_simulated, pivot_table_observed


In [67]:
# Set parameters
predictors = 'pe'
period = "test"
folds = [0, 1]

base_path = Path('/Users/sho108/Desktop/r/work/sho108/neuralhydrology_workflow/runs')

result = {
    fold: cached_eval_run(
        run_dir=base_path/f'spatial_twofold_{fold}_{predictors}_2501_181026',
        period=period,
        data_dir='/Users/sho108/Desktop/z/Data/CAMELS_AUS',
        recalculate=True
    )
    for fold in folds
}


# Extract basins and corresponding data values from the result dictionary
df = combine_datasets(result)

# Create pivot tables for simulated and observed streamflow
df_sim, df_obs = create_pivot_tables(df)

Cached data for test not found or recalculation forced. Generating and saving...
# Evaluation: 100%|██████████| 111/111 [10:05<00:00,  5.45s/it]
Cached data for test not found or recalculation forced. Generating and saving...
# Evaluation: 100%|██████████| 111/111 [10:08<00:00,  5.48s/it]


UnboundLocalError: local variable 'basins' referenced before assignment

In [65]:
df_obs

basin,102101A,104001A,105101A,105102A,105105A,107001B,108002A,108003A,112002A,112102A,...,G8140040,G8140161,G8150018,G8170002,G8190001,G8200045,G8210010,G9030124,G9030250,G9070142
date,,,,,,,,,,,,,,,,,,,,,
1981-01-01,0.019453,0.0,0.012207,0.002369,0.0,0.443266,0.300779,0.488297,2.104075,2.093312,...,0.203289,0.253205,0.283796,0.064059,0.211162,0.221410,NaN,0.0,0.019865,0.000000
1981-01-02,0.079599,0.0,0.007141,0.002101,0.0,0.087135,0.345436,0.456937,2.700644,2.281271,...,0.188329,0.061591,0.090094,0.053851,0.095057,0.292921,NaN,0.0,0.016288,0.000000
1981-01-03,0.279508,0.0,0.005141,0.001698,0.0,0.029112,0.377005,0.439916,2.316500,2.322015,...,0.141082,0.080891,0.051354,0.060525,0.104223,0.309249,NaN,0.0,0.013711,0.000000
1981-01-04,0.076994,0.0,0.007388,0.552427,0.0,0.015075,0.335896,NaN,3.275944,2.356072,...,0.149016,0.130692,0.036038,0.751139,0.051942,0.403196,NaN,0.0,0.017908,0.000000
1981-01-05,0.074554,0.0,0.002365,0.259039,0.0,0.006716,0.716899,NaN,44.533813,25.317459,...,0.166921,0.105807,0.062165,0.744106,0.067219,0.664299,NaN,0.0,0.022352,0.133489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-27,0.000561,0.0,0.000000,0.000000,0.0,0.046908,1.280213,0.373300,0.473921,1.010801,...,0.181063,0.138924,0.000000,0.203742,NaN,NaN,NaN,NaN,0.067189,0.675687
2014-12-28,0.000367,0.0,0.000000,0.000000,0.0,0.045245,1.270040,0.355366,0.494323,1.011999,...,0.194118,0.096754,0.000000,0.094707,NaN,NaN,NaN,NaN,0.092897,2.623061
2014-12-29,0.000253,0.0,0.000000,0.000000,0.0,0.043769,1.260273,0.355466,0.490394,1.008603,...,0.226663,0.081362,0.000000,0.088643,NaN,NaN,NaN,NaN,0.240223,3.592404


In [64]:
df_sim

basin,102101A,104001A,105101A,105102A,105105A,107001B,108002A,108003A,112002A,112102A,...,G8140040,G8140161,G8150018,G8170002,G8190001,G8200045,G8210010,G9030124,G9030250,G9070142
date,,,,,,,,,,,,,,,,,,,,,
1981-01-01,0.265911,0.216965,0.048364,0.137110,0.194184,-0.056539,0.239055,0.193282,1.757978,1.683916,...,0.208698,0.957334,0.391188,1.544995,2.138013,0.738056,0.729360,0.160701,0.166001,0.149155
1981-01-02,0.347635,0.137397,0.117502,0.165039,0.139375,0.288366,0.202677,0.141918,4.052753,3.502745,...,0.247251,0.828412,0.317139,1.224585,1.644261,0.806747,0.513123,0.291515,0.146758,0.062278
1981-01-03,0.343784,0.329870,0.178731,0.189646,0.236336,0.205116,0.245292,0.244418,3.301714,3.012691,...,0.230746,0.910569,0.343866,1.281734,1.146551,0.510715,0.475480,0.197406,0.175908,0.028655
1981-01-04,1.200832,0.328459,0.247062,0.374835,0.785336,0.074943,0.522357,0.396558,3.100313,2.706442,...,0.231509,0.825586,0.330974,1.182269,0.992870,0.517784,0.531774,0.165321,0.161508,0.086413
1981-01-05,1.584584,0.285171,0.935504,1.152230,3.361284,0.178913,2.078999,5.048427,16.250832,13.159662,...,0.278953,1.421491,0.443004,2.389787,1.023203,0.481109,0.453491,0.222858,0.235602,-0.003816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-27,0.192335,0.161221,0.020388,0.018520,0.182668,0.019621,0.225897,0.115126,0.358887,0.357767,...,0.346195,1.156418,0.274657,0.661922,0.380937,0.458783,0.202431,0.209140,0.194872,0.058505
2017-12-28,0.200650,0.157325,0.030261,0.028424,0.189845,0.033434,0.236732,0.111781,0.375285,0.381977,...,0.315142,0.786428,0.296435,0.575651,0.317144,0.399500,0.286197,0.172025,0.172429,0.050811
2017-12-29,0.201071,0.175764,0.020231,0.017462,0.198731,0.023966,0.242859,0.095915,0.336997,0.345601,...,0.300562,0.646290,0.282981,0.516193,0.296252,0.388151,0.250767,0.174649,0.202541,0.023728


In [ ]:
/datasets/work/d61-coastal-forecasting-wp3/work/sho108/neuralhydrology/workflows/camelaus_lstm/runs/spatial_twofold_0_e_2501_181026/train_data
/Users/sho108/projects/neuralhydrology/workflows/camelaus_lstm/petrichore/runs/spatial_twofold_0_e_2501_181026/train_data